<!--
Doc Writer email@nixdabei.de
v0.0.1, 2021-03-23
-->
[Home](../index.ipynb) / MPU-6050 Beschleunigungssensor
***
# MPU-6050 Beschleunigungssensor

* [Datenblatt](doc/MPU-6000-Datasheet.pdf) (Lokale Kopie), Quelle: [InvenSense](https://invensense.tdk.com/wp-content/uploads/2015/02/MPU-6000-Datasheet1.pdf)
* Adapted MPU-6050 driver from [Bryan Siepert (Adafruit) on Github](https://github.com/adafruit/Adafruit_CircuitPython_MPU6050.git)

## Klasse MPU6050

In [ ]:
#%serialconnect --port=COM3 # Windows with more than one COM-Port
%serialconnect # Linux / Windows with one COM-Port

In [ ]:
# g = 9.8070 : Bayern-Süd (Nieder- und Oberbayern, Schwaben)
# g = 9.8081 : Baden-Württemberg, Bayern-Nord (Franken, Oberpfalz)
# g = 9.8107 : Hessen, Nordrhein-Westfalen, Rheinland-Pfalz, Saarland, Sachsen, Thüringen
# g = 9.8130 : Berlin, Brandenburg, Bremen, Hamburg, Mecklenburg-Vorpommern, Niedersachsen, Sachsen-Anhalt, Schleswig-Holstein 
# Source: https://de.wikipedia.org/wiki/Gravitationszone

import time
import math # for pi and sqrt
from micropython import const

class MPU6050:
    # Data register and buffer
    REGID_PWR_SLEEP_1      = const(0x6B)  # Primary power/sleep control register
    REGID_SENSOR_DATA_BASE = const(0x3B)

    READ_BUFFER_SIZE = const(14)
    
    # Acceleration settings
    REGID_ACCELERATROR_CONFIG = const(0x1C)
    
    ACCELERATOR_RANGE_2_G  = const(0)  # +/-  2 g (default)
    ACCELERATOR_RANGE_4_G  = const(1)  # +/-  4 g
    ACCELERATOR_RANGE_8_G  = const(2)  # +/-  8 g
    ACCELERATOR_RANGE_16_G = const(3)  # +/- 16 g

    ACCELERATOR_SENS_SCALE_2G = 16384 # = 2^14
    GRAVITY_DEFAULT = 9.8070 # south of bavaria

    
    # Gyroscope settings
    REGID_GYROSCOPE_CONFIG = const(0x1B)
    
    GYROSCOPE_RANGE_250_DPS  = const(0)  # +/-  250 deg/s (default)
    GYROSCOPE_RANGE_500_DPS  = const(1)  # +/-  500 deg/s
    GYROSCOPE_RANGE_1000_DPS = const(2)  # +/- 1000 deg/s
    GYROSCOPE_RANGE_2000_DPS = const(3)  # +/- 2000 deg/s

    GYROSCOPE_SENS_SCALE = const(131) # math.pi/(180*131)

    
    def __init__(self, i2c, addr=0x68):
        self.iic  = i2c
        self.addr = addr

        self.iic.writeto_mem(self.addr, MPU6050.REGID_PWR_SLEEP_1, bytearray([0x80])) # Reset MPU6050 (b'\x80' not enabled in Esp8266-MicroPython: 2022-02-02)
        time.sleep( 0.1 )
        self.iic.writeto_mem(self.addr, MPU6050.REGID_PWR_SLEEP_1, bytearray([0x00])) # Disable sleep
        
        self.aDataByte = bytearray(MPU6050.READ_BUFFER_SIZE)
        self.aDataInt  = [0]   * int(MPU6050.READ_BUFFER_SIZE/2)
        self.aData     = [0.0] * int(MPU6050.READ_BUFFER_SIZE/2)
        self.aDataNorm = [0.0] * int(MPU6050.READ_BUFFER_SIZE/2)
        
        self.acceleratorSensScale = MPU6050.GRAVITY_DEFAULT/MPU6050.ACCELERATOR_SENS_SCALE_2G
        self.gyroscopeSensScale   = math.pi/(180*MPU6050.GYROSCOPE_SENS_SCALE)

        self.acceleratorNormScale = 1.0
        
    def setAcceleratorRange( self, accelRange ):
        self.iic.writeto_mem(self.addr, MPU6050.REGID_ACCELERATROR_CONFIG, bytearray([accelRange<<3]))
        # sensitifity scale: 16384, 8192, 4096, 2048 (see manual)
        self.acceleratorSensScale = MPU6050.GRAVITY_DEFAULT*(1<<accelRange)/MPU6050.ACCELERATOR_SENS_SCALE_2G

    def setGyroscopeRange( self, gyroRange ):
        self.iic.writeto_mem(self.addr, MPU6050.REGID_GYROSCOPE_CONFIG, bytearray([gyroRange<<3]))
        # sensitifity scale: 131, 65.5, 32.8, 16.4 (see manual)
        self.gyroscopeSensScale = math.pi/(180*round(MPU6050.GYROSCOPE_SENS_SCALE/(1<<gyroRange),1))
                
    @staticmethod
    def bytesToInt(byteHigh, byteLow):
        if not byteHigh & 0x80:
            return byteHigh << 8 | byteLow
        else:
            return - ( ((byteHigh^0xff) << 8) | (byteLow^0xff) + 1 ) # negativ values are in two's complement, so return  -(two's complement)
    
    def getRawDataIntArray(self):
        self.iic.readfrom_mem_into(self.addr, MPU6050.REGID_SENSOR_DATA_BASE, self.aDataByte ) # 0x3B: base address for sensor data reads

        for iIndex in range( 0, MPU6050.READ_BUFFER_SIZE, 2 ):
            self.aDataInt[ iIndex>>1 ] = MPU6050.bytesToInt( self.aDataByte[ iIndex ], self.aDataByte[ iIndex +1] )

        return self.aDataInt
    
    
    # 0, 1, 2: Accelerator: x, y, z: in m/s^2
    # 3      : Temperature: value ( convert to °C: value / 340.00 + 36.53 )
    # 4, 5, 6: Gyroscope  : x, y, z: in 1/s (omega)

    def getDataArray( self, normalize = False ):
        self.getRawDataIntArray()
        
        self.aData[0] = self.aDataInt[0] * self.acceleratorSensScale
        self.aData[1] = self.aDataInt[1] * self.acceleratorSensScale
        self.aData[2] = self.aDataInt[2] * self.acceleratorSensScale

        self.aData[3] = self.aDataInt[3]/340 + 36.53
        
        self.aData[4] = self.aDataInt[4] * self.gyroscopeSensScale
        self.aData[5] = self.aDataInt[5] * self.gyroscopeSensScale
        self.aData[6] = self.aDataInt[6] * self.gyroscopeSensScale

        if normalize == True :
            for iIndex in range( 4, 7 ):
                self.aData[iIndex] -= self.aDataNorm[iIndex]
        
            for iIndex in range( 3 ):
                self.aData[iIndex] *= self.acceleratorNormScale
                
        return self.aData   

        
    def normalize( self, iSamples = 100 ):
        for iCount in range( 100 ): # first samples are sometimes [0,0,0,0,0,0,0] : skip.
            self.getRawDataIntArray()
            
            if self.aDataInt[0] != 0 or self.aDataInt[1] != 0 or self.aDataInt[2] != 0:
                self.aDataNorm = self.getDataArray()[:]
                break
                
        # now calcualte average from the next iSamples samples:
        for iN in range( 2, iSamples+1 ):
            self.getDataArray()
            for iIndex in range(int(MPU6050.READ_BUFFER_SIZE/2)):
                self.aDataNorm[iIndex] = (self.aDataNorm[iIndex]*(iN-1) + self.aData[iIndex])/iN
          
        self.acceleratorNormScale = MPU6050.GRAVITY_DEFAULT / math.sqrt(self.aDataNorm[0]**2 + self.aDataNorm[1]**2 + self.aDataNorm[2]**2)
        self.aDataNorm[3] = 0 # Temperature needs no normalisation.      


## Sample

In [ ]:
DISPLAY_CENTER_X   = (60+128)//2
DISPLAY_LINE_HIGHT = 11
GAUGE_WIDTH_MAX    = (128-DISPLAY_CENTER_X)
GAUGE_HEIGHT       = 4

ACCELERATION_RANGE = MPU6050.ACCELERATOR_RANGE_2_G
GYROSCOPE_RANG     = MPU6050.GYROSCOPE_RANGE_250_DPS

#ACCELERATION_RANGE = MPU6050.ACCELERATOR_RANGE_4_G
#GYROSCOPE_RANG     = MPU6050.GYROSCOPE_RANGE_500_DPS


ACCELERATION_MAX   = 20 *(1<<ACCELERATION_RANGE) # +- m/s^2
OMEGA_MAX          = 4.5*(1<<GYROSCOPE_RANG)     # +- 1/s

import display
display = display.Display()
display.setCenter( 63, 0 )

from machine import I2C, Pin
import info

if   info.TYPE == "Esp8266 Croduino Nova": i2c = I2C(   scl=Pin( 4), sda=Pin( 5))
elif info.TYPE == "Esp32 NodeMCU"        : i2c = I2C(1, scl=Pin(21), sda=Pin(22))
elif info.TYPE == "Esp32 HelTec"         : i2c = I2C(1, scl=Pin(15), sda=Pin( 4))
else : print( "FATAL ERROR: Unknown controller!" )

from display import Display
display = Display(i2c)
display.setCenter( DISPLAY_CENTER_X, 0 )

mpu6050 = MPU6050(i2c)
mpu6050.normalize( 100 )

mpu6050.setAcceleratorRange( ACCELERATION_RANGE )
mpu6050.setGyroscopeRange  ( GYROSCOPE_RANG )



def printValue( strTitle, val, valMax, posY ):
    posY *= DISPLAY_LINE_HIGHT
    display.text( strTitle + "{:>5.1f}".format( val ), -DISPLAY_CENTER_X, posY )
    iVal = int(val/valMax*GAUGE_WIDTH_MAX)
    if iVal > 0 : display.fill_rect( 0,    posY,  iVal, GAUGE_HEIGHT)
    else        : display.fill_rect( iVal, posY, -iVal, GAUGE_HEIGHT)

try:
    while True:
        aData = mpu6050.getDataArray( normalize = True )

        display.clear()
        display.vline( 0,0, 64)
        display.vline( 0,29,3,0)
        printValue( "x:", aData[0], ACCELERATION_MAX, 0 )
        printValue( "y:", aData[1], ACCELERATION_MAX, 1 )
        printValue( "z:", aData[2], ACCELERATION_MAX, 2 )

        printValue( "wx:", aData[4], OMEGA_MAX, 3 )
        printValue( "wy:", aData[5], OMEGA_MAX, 4 )
        printValue( "wz:", aData[6], OMEGA_MAX, 5 )

        display.show()
except: pass
finally:
    display.clear()
    display.show ()
    print( "Done." )
